## Implementing CNN using Tensorflow

In [1]:
#Importing libraries

from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
print("All libs imported.")

All libs imported.


In [2]:
# Loading the dataset

pickle_file = './data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


#### Convolutions need the image data formatted as a cube (width by height by #channels) and labels as float 1-hot encodings.


In [3]:
# Reshaping the dataset

image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


## CNN without pooling, Convs of stride=2

In [4]:
# Finding model accuracy

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')

    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("\nStep no. :", step)
            print('Minibatch loss: %f' % l)
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized

Step no. : 0
Minibatch loss: 3.016140
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%

Step no. : 500
Minibatch loss: 0.342398
Minibatch accuracy: 81.2%
Validation accuracy: 81.2%

Step no. : 1000
Minibatch loss: 0.727077
Minibatch accuracy: 81.2%
Validation accuracy: 83.4%

Step no. : 1500
Minibatch loss: 0.841228
Minibatch accuracy: 81.2%
Validation accuracy: 84.8%

Step no. : 2000
Minibatch loss: 0.546409
Minibatch accuracy: 87.5%
Validation accuracy: 85.1%

Step no. : 2500
Minibatch loss: 0.959665
Minibatch accuracy: 75.0%
Validation accuracy: 85.8%

Step no. : 3000
Minibatch loss: 0.413193
Minibatch accuracy: 87.5%
Validation accuracy: 86.4%

Step no. : 3500
Minibatch loss: 0.278731
Minibatch accuracy: 93.8%
Validation accuracy: 86.5%

Step no. : 4000
Minibatch loss: 0.431414
Minibatch accuracy: 93.8%
Validation accuracy: 87.0%

Step no. : 4500
Minibatch loss: 1.110101
Minibatch accuracy: 68.8%
Validation accuracy: 87.3%

Step no. : 5000
Minibatch loss: 1.131758


## Using Max Pooling instead of strides for dimension reduction

In [7]:
def correct(predictions, labels):
    return np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))

In [8]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels), name = "dataset")
    tf_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels), name="labels")
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        # 28x28x1
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        # 28x28x16
        pool = tf.nn.max_pool(conv, [1,2,2,1], [1,2,2,1], padding="SAME")
        # 14x14x16
        hidden = tf.nn.relu(pool + layer1_biases)
        # 14x14x16
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        # 14x14x16
        pool = tf.nn.max_pool(conv, [1,2,2,1], [1,2,2,1], padding="SAME")
        # 7x7x16
        hidden = tf.nn.relu(pool + layer2_biases)
        # 7x7x16
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        # (7*7*16) = 784
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_dataset)
    
    # Optimizer.
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_labels, logits=logits))
    optimizer = tf.train.AdamOptimizer().minimize(loss)
  
    # Predictions for the training, validation, and test data.
    prediction = tf.nn.softmax(logits)

In [9]:
num_steps = 5001

last = -1*batch_size

with tf.Session(graph=graph) as session:
    
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps):
        
        last += batch_size
        batch_data = train_dataset[last:(last + batch_size), :, :, :]
        
        batch_labels = train_labels[last:(last + batch_size), :]
        
        feed_dict = {tf_dataset : batch_data, tf_labels : batch_labels}
        
        _, l, predictions = session.run([optimizer, loss, prediction], feed_dict=feed_dict)
        
        corr = correct(predictions, batch_labels)
        acc = int(corr)/int(batch_size) * 100
        
        if (step % 500 == 0):
            print('\nStep no: ', step)
           
            print('Minibatch loss: %f' % l)
            
            print('Minibatch accuracy: %.1f%%' % acc)
            
            num_iter = int(valid_labels.shape[0] / batch_size)
            
            offset = -1*batch_size
            
            corr = 0
            
            for i in range(num_iter):
                offset += batch_size
                batch_data = valid_dataset[offset:(offset + batch_size), :, :, :]
                batch_labels = valid_labels[offset:(offset + batch_size), :]


                feed_dict = {tf_dataset : batch_data, tf_labels : batch_labels}
        
                predictions = session.run(prediction, feed_dict=feed_dict)

                corr += correct(predictions, batch_labels)

            acc = int(corr)/int(valid_labels.shape[0])*100
            print('Validation accuracy: %.1f%%' % acc)
            
    
    print("\nTraining over!")        
    
    num_iter = int(test_labels.shape[0] / batch_size)
            
    offset = -1*batch_size
            
    corr = 0
            
    for i in range(num_iter):
        offset += batch_size
        batch_data = test_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = test_labels[offset:(offset + batch_size), :]
       
        feed_dict = {tf_dataset : batch_data, tf_labels : batch_labels}
        
        predictions = session.run(prediction, feed_dict=feed_dict)
        
        corr += correct(predictions, batch_labels)
    
    acc = int(corr)/ int(test_labels.shape[0])*100
        
    print('Test accuracy: %.1f%%' % acc)

Initialized

Step no:  0
Minibatch loss: 3.261923
Minibatch accuracy: 6.2%
Validation accuracy: 13.6%

Step no:  500
Minibatch loss: 0.377940
Minibatch accuracy: 93.8%
Validation accuracy: 80.8%

Step no:  1000
Minibatch loss: 0.633231
Minibatch accuracy: 81.2%
Validation accuracy: 84.9%

Step no:  1500
Minibatch loss: 0.440390
Minibatch accuracy: 81.2%
Validation accuracy: 86.4%

Step no:  2000
Minibatch loss: 0.356905
Minibatch accuracy: 87.5%
Validation accuracy: 87.5%

Step no:  2500
Minibatch loss: 0.955310
Minibatch accuracy: 75.0%
Validation accuracy: 87.6%

Step no:  3000
Minibatch loss: 0.428425
Minibatch accuracy: 93.8%
Validation accuracy: 87.4%

Step no:  3500
Minibatch loss: 0.170026
Minibatch accuracy: 100.0%
Validation accuracy: 88.3%

Step no:  4000
Minibatch loss: 0.330691
Minibatch accuracy: 93.8%
Validation accuracy: 88.5%

Step no:  4500
Minibatch loss: 1.174379
Minibatch accuracy: 75.0%
Validation accuracy: 88.6%

Step no:  5000
Minibatch loss: 0.980685
Minibatch a